### Load imports and run the no LID case:

In [ ]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
#%reset
import yaml
from swmm_objects import *
from swmm_read_cso_time_series import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidListFile = "PWD_wingo_RainBarrel_Only_LID_List.yaml"
################################################

client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y17m01d11_AEM_45_Subcat_RainBarrel_Only_1'

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidListFile,'r')   # Read the no LID changable parameters from file
lidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
for lid in lidList:
    lid['Number'] = 0  # initialize the lidList
starting_perm = [0]
zeroLidDocId = runswmm(lidList,swmmInpStr,thisRunCollection,starting_perm)  # run SWMM on the no LID model for baseline
#cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
#zeroLidPeak = cursor['peak']
#zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage" % zeroLidDocId

### Loop ALL subcatchment LID numbers over a range (using np.random):

In [ ]:
# Run or continue a run:
import yaml
from swmm_run_cso import *
from swmm_objects import *
from swmm_read_cso_time_series import *
from copy import deepcopy
#import numpy as np
from random import randint
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidListFile = "PWD_wingo_RainBarrel_Only_LID_List.yaml"
################################################
squareFeetInOneAcre = 43560.0
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y17m01d11_AEM_45_Subcat_RainBarrel_Only_1'
thisRunCollection = db[collection]
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
(section_names,sections) = read_inp(swmmInpStr)
modelTemp = swmm_model("ModelTemp",section_names,sections)
f = open(lidListFile,'r')   # Read the LID parameters used to calculate [LID_USAGE]
lidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
#########################
numSubcatch = 45
numLID = 1
numberOfSimulations = 150
#########################
numVariables = numSubcatch*numLID
initialRemainingTreatableArea = {}   # Dict indexed by subcatchment name string 
for lidListEntry in range(numVariables):
    thisLidInfo = lidList[lidListEntry]
    subcatName = thisLidInfo['Subcat']
    areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres   
    impPctSubcat = modelTemp.getSubcatchmentPctImperv(subcatName)        
    initialRemainingTreatableArea[subcatName] = (impPctSubcat/100.0)*areaSubcat  # INITIAL values (Acre)
#print initialRemainingTreatableArea
permList = []
greenedAcreList = []

stepSize = 0.02
fixedUpperFraction = stepSize
for i in range(0,numberOfSimulations):#number of simulations
    remainingTreatableArea = dict(initialRemainingTreatableArea)   # Dict indexed by subcatchment name string  
    lidListEntry = 0
    randomPerm = []
    greened = []
    lidListEntry = 0   
    ###############
    #randomScaleFactor = randint(3,50)
    
    fixedUpperFraction += stepSize
    if fixedUpperFraction > 1.0:
        fixedUpperFraction = 2*stepSize
    fixedLowerFraction = fixedUpperFraction - 2*stepSize
    #############    
    for lid in range(numLID):
        for subcat in range(numSubcatch):
            thisLidInfo = lidList[lidListEntry]
            #lidName = thisLidInfo['LID']
            subcatName = thisLidInfo['Subcat']
            areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres          
            contribImpervArea = thisLidInfo['ContribImpervArea']/squareFeetInOneAcre  #ft2 to Acres        
            numLidMaximum = int(remainingTreatableArea[subcatName]/contribImpervArea)
            #randomIntegerUpper = randint(1,19)
            #upperFraction = float(randomIntegerUpper)/20.0

            
            #random1 = randint(1,randomScaleFactor)
            #random2 = randint(1,randomScaleFactor)
            #if random1 > random2:
            #    randomUpper = random1
            #    randomLower = random2
            #else:
            #    randomUpper = random2
            #    randomLower = random1
            ############
            #lower = numLidMaximum/randomUpper
            #upper = numLidMaximum/randomLower 
            ############
            #fixedLowerFraction = 0.90
            #fixedUpperFraction = 0.99
            lower = int(fixedLowerFraction*numLidMaximum)
            upper = int(fixedUpperFraction*numLidMaximum)
            ############
            randomNumberLid = randint(lower,upper)
            randomPerm.append(randomNumberLid)
            greened.append(randomNumberLid*lidList[lidListEntry]['GreenedAcres'])
            remainingTreatableArea[subcatName] = remainingTreatableArea[subcatName] - \
                  randomNumberLid*contribImpervArea #Acres
            lidListEntry += 1
        #print remainingTreatableArea
    permList.append(randomPerm)
    greenedAcreList.append(greened)
#print permList 
#print greenedAcreList


runNum = 0
for perm in permList:
    greened = greenedAcreList[runNum]
    totGreenedAcres = sum(greened)
    runNum += 1
    print "Run Number %s" % runNum  
    print "Total Greened Acres %0.2f" % totGreenedAcres
    print perm
    newRunList=deepcopy(lidList)  # copy the zeroLidList into a new list
    #totalAreaTreated = 0
    for i in range(0,numVariables): 
        #lidNumber = perm[i] # random number of LIDs for this variable
        #subArea = newRunList[i]['SubcatArea'] #area of subcatchment
        newRunList[i]['Number'] = perm[i]  # random number of lids for this variable
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection,perm)
    print "stored this run in mongodb DocId = %s (zero means failed SWMM run)" % newRunDocId
    #cursor = thisRunCollection.find_one({'_id':newRunDocId})

    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"


### To display all of the runs from the database:

In [ ]:
print thisRunCollection

In [ ]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

### To Reconstitute the SWMM input file for any of the runs above

In [ ]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
#print swmmInputFileStr
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()